# 📈 Scraping de Notícias sobre Ações do site Investing.com com Selenium e Pandas

Este script realiza a coleta de notícias sobre ações, utilizando **Selenium** para navegar na página Investing.com e capturar dados dinâmicos. Os dados extraídos são processados com **Pandas** e salvos em um arquivo **CSV** para análise posterior.

### Funcionalidade:
- **Inicia o WebDriver**: Configura e executa o navegador em modo headless.
- **Captura de Notícias**: Coleta notícias da página de notícias do mercado de ações no site "Investing.com".
- **Extração de Dados**: Extrai informações de cada notícia, como título, descrição e data de publicação.
- **Conversão de Tempo**: Converte as informações de tempo relativo (ex: "5 minutos atrás") para um timestamp absoluto.
- **Armazenamento**: Organiza as notícias extraídas em um DataFrame do Pandas e as salva em um arquivo CSV.

🚀 **Objetivo**: Automatizar o processo de extração de notícias financeiras e facilitar a análise dos dados.

In [1]:
# !pip install selenium pandas

In [2]:
import os
import re
import time
import pandas as pd
import textwrap
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import undetected_chromedriver as uc

In [3]:
def convert_time_to_timestamp(time_text):
    """
    Converte uma string de tempo relativo (ex: "7 minutes ago", "2 hours ago") para um timestamp absoluto.
    """
    time_text = time_text.lower()
    try:
        if 'minute' in time_text:
            minutes_ago = int(time_text.split()[0])
            return datetime.now() - pd.to_timedelta(minutes_ago, unit='m')
        elif 'hour' in time_text:
            hours_ago = int(time_text.split()[0])
            return datetime.now() - pd.to_timedelta(hours_ago, unit='h')
        else:
            return pd.to_datetime(time_text)
    except:
        return None  # Em caso de erro, retorna None

1. Iniciar ferramenta de web scraping (Selenium)

In [4]:
def start_driver(headless=True, visible=False):
    """
    Inicia o WebDriver do Chrome com proteção contra detecção de bots (Cloudflare).

    Args:
        headless (bool): Se True, roda em modo headless (sem interface gráfica).
        visible (bool): Se True, abre janela gráfica para inspeção.

    Retorna:
        uc.Chrome: Instância do driver Chrome com camuflagem.
    """
    options = uc.ChromeOptions()

    # Define o modo de exibição
    if headless and not visible:
        options.add_argument("--headless=new")  # 'new' para melhor compatibilidade
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")

    # Define o user-agent (ajuda a parecer um navegador real)
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36")

    # Tamanho da janela
    if visible:
        options.add_argument("--window-size=1920,1080")

    # Cria o driver com detecção ofuscada
    driver = uc.Chrome(options=options)

    return driver

driver = start_driver(headless=False, visible=True)

2. Carregar a página de notícias do Investing.com

In [5]:
def load_page(driver, url: str, delay: float = 5.0):
    """
    Carrega uma página e aguarda o carregamento completo.

    Args:
        driver (webdriver.Chrome): Instância do WebDriver.
        url (str): URL a ser carregada.
        delay (float): Segundos para aguardar após o carregamento.

    Returns:
        None
    """
    driver.get(url)
    time.sleep(delay)

load_page(driver, "https://www.investing.com/news/stock-market-news")

3. Obter o nome das classes dos elementos html do site que contém as notícias

Obs: Para essa etapa é necessário inspecionar o elemento html do site, e extrair um trexo no nome da classe que contem a notícia

In [6]:
def get_class_counts(driver, substring: str):
    """
    Filtra classes de elementos cujo nome contenha a substring, imprime índice e retorna a lista.

    Args:
        driver (webdriver.Chrome): Instância do WebDriver com página carregada.
        substring (str): Substring a buscar nos nomes de classe.

    Returns:
        list[str]: Lista de nomes de classes que contêm a substring.
    """
    # Encontra elementos que tenham a substring em seu atributo class
    elements = driver.find_elements(By.CSS_SELECTOR, f"[class*='{substring}']")
    class_set = set()
    for el in elements:
        class_attr = el.get_attribute('class') or ''
        for cls in class_attr.split():
            if substring in cls:
                class_set.add(cls)

    filtered = list(class_set)
    for i, cls in enumerate(filtered):
        print(f"{i} - Classe: {cls}")
    return filtered

class_list = get_class_counts(driver, 'news-analysis-v2')

0 - Classe: news-analysis-v2_content__z0iLP
1 - Classe: news-analysis-v2_articles-container__3fFL8
2 - Classe: news-analysis-v2_info-item__dOLsl
3 - Classe: news-analysis-v2_article__wW0pT


4. Obter os textos dos elementos html referenciado pelo nome da classe

In [12]:
def get_texts_by_class(driver, class_name: str):
    """
    Retorna textos de todos os elementos que contenham a classe especificada,
    ignorando linhas que sejam apenas números, bullet points ou vazias.

    Args:
        driver (webdriver.Chrome): Instância do WebDriver com página carregada.
        class_name (str): Nome da classe CSS a ser buscada.

    Returns:
        list[str]: Lista de textos válidos dos elementos encontrados.
    """
    selector = f"[class~='{class_name}']"
    elements = driver.find_elements(By.CSS_SELECTOR, selector)
    texts = []

    for el in elements:
        text = el.text.strip()
        print(f"[DEBUG] Capturado: '{text}'") 
        # Ignora se for vazio
        if not text:
            continue
        # Ignora se for apenas número
        if re.fullmatch(r'\d+', text):
            continue
        # Ignora se for bullet point
        if text in ['•', '·', '●', '◦', '-', '–']:
            continue
        texts.append(text)

    return texts

headlines = get_texts_by_class(driver, class_list[3])

for line in headlines:
    print(line)

[DEBUG] Capturado: 'Republicans split on US credit downgrade as party’s tax bill lingers
By Bo Erickson WASHINGTON (Reuters) -Moody’s downgrade of the U.S. sovereign credit rating has elicited mixed responses among...
By
Reuters
•
35 minutes ago'
[DEBUG] Capturado: 'Trump tells Walmart to ’eat the tariffs’ instead of raising prices
By Jasper Ward WASHINGTON (Reuters) -U.S. President Donald Trump said on Saturday that Walmart (N:WMT) should "eat the tariffs"...
By
Reuters
•
1 hour ago
•
8'
[DEBUG] Capturado: 'Moody’s cuts America’s pristine credit rating, citing rising debt
By Davide Barbuscia and Pushkala Aripaka (Reuters) -Moody’s downgraded the U.S. sovereign credit rating on Friday due to concerns...
By
Reuters
•
3 hours ago
•
19'
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Capturado: ''
[DEBUG] Captur

5. Obter os textos relevantes para a classificação do sentimento, como o título, a descrição da notícia e a data que ela foi postada

Obs: **Análise** a linhas da lista obtida anteriormente e **ajuste** os parametros de acordo para a leitura correta

In [ ]:
def convert_time_to_timestamp(time_text):
    """
    Converte uma string de tempo relativo (ex: "7 minutes ago", "2 hours ago") para um timestamp absoluto.
    """
    time_text = time_text.lower()
    try:
        if 'minute' in time_text:
            minutes_ago = int(time_text.split()[0])
            return datetime.now() - pd.to_timedelta(minutes_ago, unit='m')
        elif 'hour' in time_text:
            hours_ago = int(time_text.split()[0])
            return datetime.now() - pd.to_timedelta(hours_ago, unit='h')
        else:
            return pd.to_datetime(time_text)
    except:
        return None  # Em caso de erro, retorna None

def parse_news_items(headlines: list[str], start_line: int, end_line: int, lines_per_news: int = 2,
                     titulo_offset: int = 0, descricao_offset: int = 1, data_offset: int | None = None) -> list[dict]:
    """
    Remove os 'start_line' primeiros e 'end_line' últimos elementos da lista e agrupa de n em n, extraindo título, descrição e data.

    Args:
        headlines (list[str]): Lista de textos capturados pelo get_texts_by_class.
        start_line (int): Quantos itens remover do início.
        end_line (int): Quantos itens remover do final.
        lines_per_news (int): Tamanho do agrupamento (padrão 2 = título + descrição).
        titulo_offset (int): Índice relativo dentro do grupo para o título.
        descricao_offset (int): Índice relativo dentro do grupo para a descrição.
        data_offset (int | None): Índice relativo dentro do grupo para a data. Se None, a data não será extraída.

    Returns:
        list[dict]: Lista de dicionários com 'titulo', 'descricao' e 'data' (se disponível).
    """
    trimmed = headlines[start_line:len(headlines) - end_line]
    noticias = []

    for i in range(0, len(trimmed) - (lines_per_news - 1), lines_per_news):
        grupo = trimmed[i:i + lines_per_news]
        if len(grupo) == lines_per_news:
            try:
                titulo = grupo[titulo_offset].strip()
                descricao = grupo[descricao_offset].strip()
                data = None

                if data_offset is not None and 0 <= data_offset < len(grupo):
                    raw_data = grupo[data_offset].strip()
                    data = convert_time_to_timestamp(raw_data)

                noticias.append({
                    "titulo": titulo,
                    "descricao": descricao,
                    "data": data
                })
            except IndexError:
                continue  # pula grupos mal formados

    return noticias

news = parse_news_items(headlines, start_line=0, end_line=0, lines_per_news=3, title_offset=0, description_offset=1)
news

[{'titulo': 'Republicans split on US credit downgrade as party’s tax bill lingers\nBy Bo Erickson WASHINGTON (Reuters) -Moody’s downgrade of the U.S. sovereign credit rating has elicited mixed responses among...\nBy\nReuters\n•\n13 minutes ago',
  'descricao': 'Trump tells Walmart to ’eat the tariffs’ instead of raising prices\nBy Jasper Ward WASHINGTON (Reuters) -U.S. President Donald Trump said on Saturday that Walmart (N:WMT) should "eat the tariffs"...\nBy\nReuters\n•\n1 hour ago\n•\n8',
  'data': None}]

In [10]:
def extract_text_string(html_string, keyword, start_char, end_char, division_index: int=1):
    """
    Extrai um trecho de texto de uma string HTML com base em uma palavra-chave e os caracteres delimitadores.

    Args:
        html_string (str): A string HTML de onde extrair o texto.
        keyword (str): Palavra-chave para localizar o ponto de extração.
        start_char (str): Caractere de início do trecho a ser extraído.
        end_char (str): Caractere de fim do trecho a ser extraído.
        division_index (int, opcional): Índice para dividir a string HTML (padrão é 1).

    Retorna:
        str: O texto extraído, ou None se a extração falhar.
    """
    try:
        # Divide o HTML pela palavra-chave
        parts = html_string.split(keyword)
        # print(f"Parts[1] ({len(parts)}) :", parts[1])
        
        # Verifica se a palavra-chave está no HTML
        if len(parts) < 2:
            print(f'({len(parts)})\n\n({keyword})\n{parts}\n\n')
            return None
        
        # Pega a segunda parte da divisão
        text_part = parts[division_index]
        
        # Encontra o índice do caractere inicial
        start_index = text_part.find(start_char)
        
        # Verifica se o caractere inicial foi encontrado
        if start_index == -1:
            return None
        
        # Ajusta o índice inicial para começar após o caractere inicial
        start_index += len(start_char)
        
        # Encontra o índice do caractere final a partir do índice inicial
        end_index = text_part.find(end_char, start_index)
        
        # Verifica se o caractere final foi encontrado
        if end_index == -1:
            return None
        
        # Extrai o texto entre o caractere inicial e o caractere final
        extracted_text = text_part[start_index:end_index]
        
        return extracted_text.strip()
    except Exception as e:
        print(f"Erro ao extrair texto: {e}")
        return None

In [11]:
# Função para extrair dados de um item de notícia
def extract_news_item_data(item):
    """
    Extrai dados (link, título, descrição, data de publicação) de um item de notícia.

    Args:
        item (str): String HTML representando um item de notícia.

    Retorna:
        dict: Dicionário contendo 'Link', 'Title', 'Description' e 'PublishDate'.
    """
    # print(f'Item:\n\n{textwrap.indent(item.get_attribute("innerHTML"), prefix='    ')}\n\n')
    try:        
        # Extraindo o link do artigo
        link = extract_text_string(item, "article-title-link", 'href="', '"', division_index=0)
        print(f'Article Link: {link}')

        # Extraindo o título do artigo
        title = extract_text_string(item, "article-title-link", ">", "<")
        print(f'Article Title: {title}')

        # Extraindo a descrição do artigo
        description = extract_text_string(item, "article-description", ">", "<")
        print(f'Article Description: {description}')

        # Extraindo a data de publicação
        publish_date = extract_text_string(item, "article-publish-date", 'datetime="', '"')
        print(f'Publish Date: {publish_date}')

        return {
            'Link': link,
            'Title': title,
            'Description': description,
            'PublishDate': publish_date
        }
    except Exception as e:
        print(f"Erro ao processar item: {e}")
        return None

In [12]:
# Função para extrair os dados das notícias
def extract_news_data(news_items):
    """
    Extrai dados de uma lista de itens de notícias e os organiza em um DataFrame.

    Args:
        news_items (list): Lista de strings HTML contendo os itens de notícias.

    Retorna:
        pd.DataFrame: DataFrame contendo os dados extraídos (link, título, descrição, data de publicação).
    """
    # Lista para armazenar os dados
    news_data = []

    # Itera sobre os itens de notícias
    for item in news_items:
        if item.startswith('<li class="list_list__item__dwS6E !mt-0 border-t'):
            news_item_data = extract_news_item_data(item)
            if news_item_data:
                news_data.append(news_item_data)

    # Cria um DataFrame com os dados extraídos
    news_df = pd.DataFrame(news_data)

    return news_df

news_df = extract_news_data(news_items)
news_df.to_csv('news_data.csv', index=False)
print("Dados salvos em 'news_data.csv'")

Dados salvos em 'news_data.csv'


In [13]:
def scrape_and_save_news():
    """
    Função principal que executa o scraping das notícias, processa os dados e os salva em um arquivo CSV.

    Salva os dados em 'news_data.csv', adicionando dados se o arquivo já existir.
    """
    driver = start_driver()
    news_df = extract_news_data(driver)
    driver.quit()  # Fecha o navegador
    
    # Verifica se o arquivo já existe
    file_exists = os.path.isfile('news_data.csv')
    
    # Salva o DataFrame em um arquivo CSV, adicionando dados se o arquivo já existir
    news_df.to_csv('news_data.csv', mode='a', header=not file_exists, index=False)
    print("Dados adicionados em 'news_data.csv'")
    
# Chama a função principal
# scrape_and_save_news()